In [433]:
#!pip install yfinance --upgrade
#!pip install talib-binary
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib

In [434]:
import os
import sys
import copy
import datetime
import time

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from talib import MA_Type

**1) Define Tickers List, Backtest Date Range and Data Retrival Intervals**

In [435]:
# define list of tickers to backtest
ticker_list = ['BTC-USD', 'BTC-ETH']

# define date range
start_date = '2022-03-20'
end_date = '2022-12-31'
interval = '1h'

# initial capital in USD
initial_capital = 1000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [436]:
BTC = yf.download('BTC-USD', start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
BTC['200EMA'] = ta.EMA(BTC['Close'], timeperiod=200) 
BTC['RSI'] = ta.RSI(BTC['Close'], timeperiod=14)
BTC['MACD'], BTC['MACD_signal'], BTC['MACD_hist'] = ta.MACD(BTC['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
# add/drop columns and clean up
#globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
BTC = BTC.round(4).dropna()
BTC = BTC.reset_index()
BTC['Position'] = 0
BTC[['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN

In [437]:
ETH = yf.download('ETH-USD', start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
ETH['200EMA'] = ta.EMA(ETH['Close'], timeperiod=200) 
ETH['RSI'] = ta.RSI(ETH['Close'], timeperiod=14)
ETH['MACD'], ETH['MACD_signal'], ETH['MACD_hist'] = ta.MACD(ETH['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
# add/drop columns and clean up
#globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
ETH = ETH.round(4).dropna()
ETH = ETH.reset_index()
ETH['Position'] = 0
ETH[['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN

In [438]:
ADA = yf.download('ADA-USD', start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
ADA['200EMA'] = ta.EMA(ADA['Close'], timeperiod=200) 
ADA['RSI'] = ta.RSI(ADA['Close'], timeperiod=14)
ADA['MACD'], ADA['MACD_signal'], ADA['MACD_hist'] = ta.MACD(ADA['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
# add/drop columns and clean up
#globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
ADA = ADA.round(4).dropna()
ADA = ADA.reset_index()
ADA['Position'] = 0
ADA[['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN

In [439]:
SOL = yf.download('SOL-USD', start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
SOL['200EMA'] = ta.EMA(SOL['Close'], timeperiod=200) 
SOL['RSI'] = ta.RSI(SOL['Close'], timeperiod=14)
SOL['MACD'], SOL['MACD_signal'], SOL['MACD_hist'] = ta.MACD(SOL['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
# add/drop columns and clean up
#globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
SOL = SOL.round(4).dropna()
SOL = SOL.reset_index()
SOL['Position'] = 0
SOL[['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN

In [440]:
EOS = yf.download('EOS-USD', start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
EOS['200EMA'] = ta.EMA(EOS['Close'], timeperiod=200) 
EOS['RSI'] = ta.RSI(EOS['Close'], timeperiod=14)
EOS['MACD'], EOS['MACD_signal'], EOS['MACD_hist'] = ta.MACD(EOS['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
# add/drop columns and clean up
#globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
EOS = EOS.round(4).dropna()
EOS = EOS.reset_index()
EOS['Position'] = 0
EOS[['Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN

**3) Building the DataFrame List, Perform Backtesting**

In [441]:
df_list = [BTC, ETH, ADA, SOL, EOS]

**Back Testing** </b>


*   Backtesting of Trade Logic across all tickers and in the designated timeframe
*   Backtesting for Trade strategy with simulated PNL



In [442]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [443]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

In [444]:
# defining the global list for storing output
global macdema_trade_list
macdema_trade_list = []

pos_size = round(initial_capital/len(ticker_list),2)

In [445]:
def backtest_macdema(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.98
  profit_target = initial_equity_value * 1.2

  for i in range(len(df)):
    now_date        = df.loc[i, 'Datetime']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_MACD > now_MACD_signal) and (now_MACD > 0) and (now_open < now_200EMA)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size/open_price
      #buy_commission = (0.0049 + 0.005) * lot_size
      #initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_equity_value = round((lot_size * open_price),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Equity Value'] = initial_equity_value
      macdema_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_MACD < now_MACD_signal) or (now_MACD < 0))) or \
      ((pos_opened == True) and (profit_target > initial_equity_value)) or \
      ((pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Datetime'])) or \
      ((pos_opened == True) and (now_open*lot_size < stop_loss)):
#      ((pos_opened == True) and (now_open*lot_size < stop_loss)) or \
#      ((pos_opened == True) and (now_open*lot_size > profit_target)):
      pos_opened = False
      close_price = now_open
      #sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      #pnl = round(((close_price - initial_buy_price) * lot_size) - sell_commission,2)
      pnl = round(((close_price - initial_buy_price) * lot_size),2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      macdema_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].mean().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [446]:
macdema_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_macdema(df)
    macdema_result_list.append(test_result)

    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = macdema_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    macdema_result_df = pd.DataFrame(macdema_result_list)
    macdema_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [447]:
# PNL for MACD EMA Strategy

#measure_date = df_list[1].loc[1, 'Datetime']
#end_date = df_list[1].loc[df.index[-1], 'Datetime'] 
print('Trading Strategy: MACD + 200 EMA')
#print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((macdema_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = macdema_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (macdema_result_df['No of Trade'].sum()))
print('Total PNL: USD', macdema_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(macdema_result_df[macdema_result_df['No of Trade'] > 0]['Win Rate %'].mean(), 2), '%')
macdema_result_df. at[0,'Ticker'] = 'BTC'
macdema_result_df. at[1,'Ticker'] = 'ETH'
macdema_result_df. at[2,'Ticker'] = 'ADA'
macdema_result_df. at[3,'Ticker'] = 'SOL'
macdema_result_df. at[4,'Ticker'] = 'EOS'
macdema_result_df[macdema_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: MACD + 200 EMA
Total No of Trades Executed: 461
Total PNL: USD 2696.86
Required Capital: USD 1000
ROI:  269.69 %
Avg Win Rate:  62.15 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
3,SOL,97,741.70,7.65,11.27,11.56,64.95
2,ADA,97,525.54,5.42,8.84,11.53,61.86
4,EOS,81,512.32,6.32,9.69,12.37,56.79
1,ETH,98,491.60,5.02,8.23,11.20,61.22
0,BTC,88,425.70,4.84,6.99,12.06,65.91
